In [1]:
from __windmodel5x5DAC__ import glob_all, save_models, WindLossmodel, define_composite_wind_model
from __windmodel5x5DAC__ import define_generator, define_discriminator, define_composite_model, train
from tensorflow.keras import backend as K

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().disabled = True

import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
tf.config.optimizer.set_jit(True)

In [2]:
resnet_n = 9 ; epochs = 500; batch = 1

home = '/media/disk3/weisting/CycleGan/Experiment5/'

dataA_norm = np.load('new_dataA_norm.npy')
dataB_norm = np.load('new_dataB_norm.npy')
print('Loaded', dataA_norm.shape, dataB_norm.shape)

EPA73_testing_PM25 = np.load(home+'348X204/EPA73_Testing_348X204_PM25.npy', mmap_mode='r')[:3000]

Train_Direc = np.concatenate([np.load('CWBEPA_Content_WIND_DIREC_graph_Training_K3.npy'),
                             np.load('CWBEPA_Content_WIND_DIREC_graph_Training_K3.npy')])

Train_Speed = np.concatenate([np.load('CWBEPA_Content_WIND_SPEED_graph_Training_K3.npy'),
                             np.load('CWBEPA_Content_WIND_SPEED_graph_Training_K3.npy')])

station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
extract5_list = np.load('RandomTesting/extract5_list.npy')
extract10_list = np.load('RandomTesting/extract10_list.npy')
extract15_list = np.load('RandomTesting/extract15_list.npy')
extract20_list = np.load('RandomTesting/extract20_list.npy')

ex20_KNNhalf = np.load('ex20_KNN_half.npy', mmap_mode='r')
ex20_KNN_norm = np.load('ex20_KNN_norm.npy', mmap_mode='r')[:,:3000,:,:]

model_path = 'CWBEPA_Graph_RMSprop_Res'+str(resnet_n)+'_CycleGan_5x5DAwind/'
if not os.path.exists(model_path+'result/'):os.makedirs(model_path+'result/')

Loaded (3194, 348, 204, 1) (3194, 348, 204, 1)


In [ ]:
K.clear_session()
image_shape = dataB_norm[0].shape
g_model_AtoB = define_generator(image_shape,resnet_n)
print('generator: A -> B')
g_model_BtoA = define_generator(image_shape,resnet_n)
print('generator: B -> A')
d_model_A = define_discriminator(image_shape, d_lr=5e-5)
print('discriminator: A -> [real/fake]')
d_model_B = define_discriminator(image_shape, d_lr=5e-5)
print('discriminator: B -> [real/fake]')
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, g_lr = 1e-3)
print('composite: A -> B -> [real/fake, A]')
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, g_lr = 1e-3)
print('composite: B -> A -> [real/fake, B]')

WindLoss_model = WindLossmodel()
print('WindLoss_model')
c_model_wind = define_composite_wind_model(g_model_AtoB, WindLoss_model, image_shape, g_lr= 1e-5)
print('composite_wind_model')

train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, c_model_wind,
      dataA_norm, dataB_norm, model_path, EPA73_testing_PM25, Train_Direc, Train_Speed,
      station_coordinate, extract5_list, extract10_list, extract15_list, extract20_list, ex20_KNN_norm, 
      ex20_KNNhalf, n_epochs=epochs, n_batch=batch, save_epochs = 1)

print('train models done')

generator: A -> B
generator: B -> A
discriminator: A -> [real/fake]
discriminator: B -> [real/fake]
composite: A -> B -> [real/fake, A]
composite: B -> A -> [real/fake, B]
WindLoss_model
composite_wind_model
3194
>00001, dALoss[1.647,1.307] dAAcc[0.179,0.673] dBLoss[0.809,2.240] dBAcc[0.371,0.513] g[22.373,24.693] wind[69.912]
>00002, dALoss[4.834,2.177] dAAcc[0.480,0.700] dBLoss[8.601,4.540] dBAcc[0.667,0.269] g[30.961,20.202] wind[75.698]
>00003, dALoss[1.163,6.453] dAAcc[0.498,0.434] dBLoss[3.435,2.146] dBAcc[0.524,0.546] g[30.951,33.217] wind[133.725]
>00004, dALoss[4.723,0.645] dAAcc[0.285,0.583] dBLoss[1.457,1.224] dBAcc[0.575,0.546] g[22.798,24.631] wind[56.784]
>00005, dALoss[2.432,1.305] dAAcc[0.643,0.502] dBLoss[0.641,3.539] dBAcc[0.477,0.454] g[25.488,23.092] wind[151.501]
>00006, dALoss[0.455,5.216] dAAcc[0.492,0.599] dBLoss[0.223,3.179] dBAcc[0.543,0.668] g[26.833,28.700] wind[48.103]
>00007, dALoss[0.305,3.566] dAAcc[0.549,0.404] dBLoss[1.077,1.333] dBAcc[0.637,0.410] g[2

>00070, dALoss[0.115,0.124] dAAcc[0.738,0.889] dBLoss[0.181,0.043] dBAcc[0.816,0.966] g[15.947,14.977] wind[58.639]
>00071, dALoss[0.263,0.259] dAAcc[0.794,0.856] dBLoss[0.425,0.049] dBAcc[0.698,0.986] g[16.314,16.102] wind[80.961]
>00072, dALoss[0.496,0.270] dAAcc[0.562,0.861] dBLoss[0.227,0.087] dBAcc[0.835,0.864] g[14.443,14.578] wind[111.984]
>00073, dALoss[0.305,0.195] dAAcc[0.719,0.844] dBLoss[0.086,0.058] dBAcc[0.929,0.911] g[16.248,15.556] wind[59.482]
>00074, dALoss[0.270,0.084] dAAcc[0.842,0.978] dBLoss[0.305,0.058] dBAcc[0.636,0.973] g[15.117,14.716] wind[60.111]
>00075, dALoss[0.172,0.037] dAAcc[0.736,0.969] dBLoss[0.308,0.113] dBAcc[0.910,0.826] g[16.500,16.073] wind[59.895]
>00076, dALoss[0.164,0.105] dAAcc[0.813,0.877] dBLoss[0.292,0.110] dBAcc[0.896,0.974] g[15.767,14.794] wind[74.248]
>00077, dALoss[0.052,0.076] dAAcc[0.986,0.824] dBLoss[0.278,0.048] dBAcc[0.658,0.937] g[15.704,15.728] wind[105.115]
>00078, dALoss[0.102,0.139] dAAcc[0.979,0.663] dBLoss[0.208,0.136] dBA

>00141, dALoss[0.115,0.016] dAAcc[0.720,0.980] dBLoss[0.036,0.041] dBAcc[0.984,0.950] g[12.373,11.230] wind[65.837]
>00142, dALoss[0.181,0.056] dAAcc[0.988,0.983] dBLoss[0.032,0.035] dBAcc[0.960,1.000] g[14.736,14.365] wind[128.303]
>00143, dALoss[0.096,0.015] dAAcc[0.927,0.992] dBLoss[0.026,0.010] dBAcc[0.978,0.997] g[14.984,14.978] wind[83.813]
>00144, dALoss[0.149,0.052] dAAcc[0.811,0.907] dBLoss[0.064,0.084] dBAcc[0.930,0.895] g[13.972,13.841] wind[78.025]
>00145, dALoss[0.065,0.066] dAAcc[0.916,0.967] dBLoss[0.087,0.043] dBAcc[0.821,0.992] g[12.363,12.460] wind[125.459]
>00146, dALoss[0.097,0.019] dAAcc[0.856,0.986] dBLoss[0.067,0.022] dBAcc[0.975,0.999] g[14.400,14.007] wind[56.304]
>00147, dALoss[0.092,0.085] dAAcc[0.857,0.893] dBLoss[0.036,0.024] dBAcc[0.966,0.933] g[13.046,12.271] wind[123.048]
>00148, dALoss[0.038,0.069] dAAcc[0.957,0.993] dBLoss[0.132,0.014] dBAcc[0.926,0.993] g[11.948,11.459] wind[130.589]
>00149, dALoss[0.034,0.013] dAAcc[0.998,0.998] dBLoss[0.087,0.020] d

>00212, dALoss[0.069,0.046] dAAcc[0.999,1.000] dBLoss[0.059,0.015] dBAcc[0.941,0.940] g[13.938,14.104] wind[109.853]
>00213, dALoss[0.020,0.019] dAAcc[0.967,0.936] dBLoss[0.132,0.073] dBAcc[0.924,1.000] g[12.492,11.695] wind[89.090]
>00214, dALoss[0.041,0.025] dAAcc[0.950,0.938] dBLoss[0.040,0.013] dBAcc[0.927,1.000] g[10.511,10.607] wind[102.149]
>00215, dALoss[0.025,0.045] dAAcc[0.946,0.997] dBLoss[0.025,0.019] dBAcc[0.989,0.983] g[13.433,12.994] wind[80.250]
>00216, dALoss[0.027,0.035] dAAcc[0.990,0.958] dBLoss[0.017,0.005] dBAcc[0.987,1.000] g[13.584,13.502] wind[53.320]
>00217, dALoss[0.037,0.012] dAAcc[0.934,0.995] dBLoss[0.027,0.004] dBAcc[0.989,1.000] g[11.979,10.697] wind[114.815]
>00218, dALoss[0.022,0.006] dAAcc[0.979,0.999] dBLoss[0.028,0.003] dBAcc[0.975,1.000] g[14.134,14.166] wind[113.694]
>00219, dALoss[0.050,0.017] dAAcc[0.990,1.000] dBLoss[0.030,0.007] dBAcc[0.971,0.998] g[11.925,11.606] wind[116.270]
>00220, dALoss[0.020,0.014] dAAcc[0.976,0.941] dBLoss[0.019,0.013] 

>00283, dALoss[0.031,0.013] dAAcc[0.990,0.969] dBLoss[0.015,0.005] dBAcc[0.993,1.000] g[12.056,12.055] wind[91.812]
>00284, dALoss[0.064,0.021] dAAcc[0.916,0.955] dBLoss[0.015,0.002] dBAcc[0.981,1.000] g[9.580,10.425] wind[64.183]
>00285, dALoss[0.045,0.012] dAAcc[0.971,1.000] dBLoss[0.020,0.015] dBAcc[1.000,1.000] g[9.769,9.607] wind[68.215]
>00286, dALoss[0.046,0.031] dAAcc[1.000,1.000] dBLoss[0.023,0.006] dBAcc[0.979,0.999] g[11.630,11.460] wind[90.478]
>00287, dALoss[0.013,0.037] dAAcc[1.000,0.918] dBLoss[0.059,0.084] dBAcc[0.907,0.924] g[10.884,10.577] wind[66.698]
>00288, dALoss[0.013,0.006] dAAcc[0.985,1.000] dBLoss[0.044,0.008] dBAcc[0.959,1.000] g[10.173,10.269] wind[82.107]
>00289, dALoss[0.014,0.005] dAAcc[0.986,0.995] dBLoss[0.017,0.007] dBAcc[0.996,1.000] g[9.555,8.965] wind[68.547]
>00290, dALoss[0.024,0.024] dAAcc[1.000,1.000] dBLoss[0.029,0.020] dBAcc[0.939,0.914] g[10.068,10.447] wind[77.660]
>00291, dALoss[0.019,0.007] dAAcc[0.980,0.991] dBLoss[0.031,0.032] dBAcc[0.99

...

>15294, dALoss[0.007,0.009] dAAcc[1.000,0.999] dBLoss[0.049,0.079] dBAcc[0.938,0.839] g[0.945,1.536] wind[101.775]
>15295, dALoss[0.009,0.020] dAAcc[1.000,1.000] dBLoss[0.049,0.095] dBAcc[0.928,0.824] g[0.992,1.717] wind[62.717]
>15296, dALoss[0.009,0.011] dAAcc[1.000,1.000] dBLoss[0.118,0.100] dBAcc[0.556,0.647] g[1.187,1.705] wind[96.726]
>15297, dALoss[0.014,0.042] dAAcc[0.998,1.000] dBLoss[0.256,0.122] dBAcc[0.033,0.573] g[1.695,2.755] wind[74.870]
>15298, dALoss[0.040,0.031] dAAcc[1.000,0.942] dBLoss[0.150,0.173] dBAcc[0.348,0.374] g[0.883,1.196] wind[67.766]
>15299, dALoss[0.047,0.020] dAAcc[0.940,0.985] dBLoss[0.278,0.136] dBAcc[0.028,0.498] g[1.324,1.827] wind[89.021]
>15300, dALoss[0.009,0.081] dAAcc[1.000,0.858] dBLoss[0.217,0.307] dBAcc[0.094,0.124] g[0.853,1.342] wind[78.836]
>15301, dALoss[0.075,0.017] dAAcc[0.898,0.995] dBLoss[0.130,0.051] dBAcc[0.519,0.926] g[1.705,2.552] wind[72.061]
>15302, dALoss[0.010,0.016] dAAcc[0.991,0.986] dBLoss[0.128,0.088] dBAcc[0.557,0.834] g

>15366, dALoss[0.007,0.016] dAAcc[1.000,0.996] dBLoss[0.073,0.066] dBAcc[0.929,0.919] g[1.562,1.851] wind[111.677]
>15367, dALoss[0.040,0.015] dAAcc[0.973,0.997] dBLoss[0.133,0.219] dBAcc[0.554,0.178] g[0.835,1.957] wind[75.954]
>15368, dALoss[0.012,0.013] dAAcc[0.999,1.000] dBLoss[0.130,0.131] dBAcc[0.574,0.523] g[1.061,1.695] wind[55.686]
>15369, dALoss[0.005,0.009] dAAcc[1.000,1.000] dBLoss[0.036,0.076] dBAcc[0.968,0.795] g[1.604,2.245] wind[77.527]
>15370, dALoss[0.016,0.100] dAAcc[0.999,0.691] dBLoss[0.068,0.079] dBAcc[0.898,0.881] g[1.091,1.196] wind[108.641]
>15371, dALoss[0.039,0.019] dAAcc[0.995,1.000] dBLoss[0.112,0.052] dBAcc[0.621,0.969] g[1.135,2.042] wind[56.078]
>15372, dALoss[0.012,0.011] dAAcc[1.000,0.996] dBLoss[0.070,0.067] dBAcc[0.864,0.896] g[1.611,2.169] wind[74.505]
>15373, dALoss[0.015,0.037] dAAcc[0.989,0.949] dBLoss[0.024,0.040] dBAcc[0.957,0.934] g[1.388,1.813] wind[104.635]
>15374, dALoss[0.014,0.016] dAAcc[0.973,0.986] dBLoss[0.154,0.024] dBAcc[0.356,0.989]

>15438, dALoss[0.015,0.035] dAAcc[0.988,0.959] dBLoss[0.083,0.074] dBAcc[0.908,0.832] g[0.967,1.206] wind[44.319]
>15439, dALoss[0.037,0.025] dAAcc[0.957,0.999] dBLoss[0.018,0.049] dBAcc[0.986,0.906] g[1.166,2.286] wind[68.150]
>15440, dALoss[0.015,0.040] dAAcc[1.000,0.993] dBLoss[0.174,0.210] dBAcc[0.334,0.127] g[0.953,1.517] wind[79.223]
>15441, dALoss[0.011,0.016] dAAcc[1.000,0.998] dBLoss[0.109,0.084] dBAcc[0.697,0.798] g[0.917,1.764] wind[90.627]
>15442, dALoss[0.007,0.014] dAAcc[1.000,0.995] dBLoss[0.119,0.029] dBAcc[0.579,0.980] g[1.745,2.113] wind[57.808]
>15443, dALoss[0.007,0.015] dAAcc[0.997,0.993] dBLoss[0.055,0.094] dBAcc[0.915,0.771] g[1.062,1.985] wind[109.387]
>15444, dALoss[0.010,0.018] dAAcc[0.990,0.986] dBLoss[0.070,0.100] dBAcc[0.903,0.667] g[0.905,1.432] wind[89.225]
>15445, dALoss[0.033,0.055] dAAcc[0.951,0.888] dBLoss[0.095,0.160] dBAcc[0.707,0.330] g[1.505,1.974] wind[125.082]
>15446, dALoss[0.039,0.012] dAAcc[0.941,0.996] dBLoss[0.144,0.147] dBAcc[0.313,0.385] 

>15510, dALoss[0.025,0.096] dAAcc[0.971,0.778] dBLoss[0.157,0.124] dBAcc[0.299,0.574] g[0.867,1.221] wind[67.805]
>15511, dALoss[0.053,0.008] dAAcc[0.907,0.995] dBLoss[0.021,0.075] dBAcc[0.996,0.865] g[1.000,2.253] wind[60.242]
>15512, dALoss[0.012,0.017] dAAcc[0.993,0.989] dBLoss[0.210,0.119] dBAcc[0.200,0.534] g[1.312,1.674] wind[92.420]
>15513, dALoss[0.011,0.017] dAAcc[1.000,0.994] dBLoss[0.134,0.025] dBAcc[0.417,0.990] g[2.289,3.626] wind[75.298]
>15514, dALoss[0.065,0.082] dAAcc[0.888,0.804] dBLoss[0.084,0.026] dBAcc[0.781,0.993] g[1.942,2.409] wind[65.585]
>15515, dALoss[0.030,0.016] dAAcc[0.990,0.995] dBLoss[0.067,0.137] dBAcc[0.896,0.495] g[1.056,2.105] wind[138.546]
>15516, dALoss[0.004,0.012] dAAcc[1.000,0.996] dBLoss[0.043,0.094] dBAcc[0.988,0.741] g[1.291,2.070] wind[108.659]
>15517, dALoss[0.026,0.091] dAAcc[0.993,0.764] dBLoss[0.074,0.029] dBAcc[0.845,0.950] g[1.502,1.693] wind[72.400]
>15518, dALoss[0.019,0.015] dAAcc[0.996,1.000] dBLoss[0.058,0.113] dBAcc[0.914,0.673] 

>15582, dALoss[0.009,0.012] dAAcc[0.999,0.993] dBLoss[0.035,0.123] dBAcc[0.970,0.565] g[1.027,1.602] wind[110.029]
>15583, dALoss[0.016,0.016] dAAcc[0.999,0.998] dBLoss[0.057,0.107] dBAcc[0.910,0.634] g[1.161,1.989] wind[113.890]
>15584, dALoss[0.008,0.031] dAAcc[1.000,0.979] dBLoss[0.115,0.056] dBAcc[0.587,0.935] g[1.238,1.545] wind[66.611]
>15585, dALoss[0.033,0.032] dAAcc[0.974,0.981] dBLoss[0.140,0.143] dBAcc[0.492,0.470] g[1.106,1.843] wind[105.445]
>15586, dALoss[0.042,0.038] dAAcc[0.990,0.977] dBLoss[0.197,0.236] dBAcc[0.151,0.051] g[0.683,1.547] wind[48.881]
>15587, dALoss[0.041,0.057] dAAcc[0.968,0.928] dBLoss[0.201,0.105] dBAcc[0.150,0.718] g[0.937,1.761] wind[66.507]
>15588, dALoss[0.100,0.032] dAAcc[0.709,0.946] dBLoss[0.097,0.083] dBAcc[0.721,0.802] g[2.159,2.634] wind[65.849]
>15589, dALoss[0.021,0.007] dAAcc[1.000,0.998] dBLoss[0.042,0.075] dBAcc[0.960,0.833] g[1.180,1.529] wind[66.190]
>15590, dALoss[0.032,0.035] dAAcc[0.965,0.960] dBLoss[0.055,0.112] dBAcc[0.907,0.747]